In [62]:
import pandas as pd
import os
import regex as re
import copy

In [68]:
# Read the crop.csv file into a DataFrame
colab = False

if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    data = pd.read_csv('/content/drive/MyDrive/CS418/Data_Bandits_CS418/data/crop_data/corn.csv')
else:
    data = pd.read_csv('../data/crop_data/corn.csv')

In [85]:
data.head()

# Create a new empty DataFrame
cleaned_df = pd.DataFrame()


In [127]:
cleaned_df = copy.deepcopy(data)


cleaned_df["PriceIsNan"] = data["PRICE RECEIVED in $ / BU"].apply(lambda x: not bool(re.match(r'^[0-9]', str(x))))
cleaned_df["AreaisNan"] = data["AREA PLANTED in ACRES"].apply(lambda x: not bool(re.match(r'^[0-9]', str(x))))

# Filter rows where "AREA PLANTED in ACRES" is greater than 100,000
# First remove commas from the values and convert to numeric
cleaned_df = data[cleaned_df['AreaisNan'] == False]
cleaned_df["AREA PLANTED in ACRES"] = cleaned_df["AREA PLANTED in ACRES"].str.replace(',', '').astype(int)
cleaned_df = cleaned_df[cleaned_df["AREA PLANTED in ACRES"] > 3000000]

C:\Users\mansu\AppData\Local\Temp\ipykernel_14436\1002942844.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df["AREA PLANTED in ACRES"] = cleaned_df["AREA PLANTED in ACRES"].str.replace(',', '').astype(int)


In [128]:
cleaned_df.head()

,YEAR,LOCATION,STATE ANSI,ASD CODE,COUNTY ANSI,REFERENCE PERIOD,COMMODITY,AREA HARVESTED in ACRES,PRODUCTION in BU,YIELD in BU / ACRE,AREA PLANTED in ACRES,PRODUCTION in $,PRODUCTION in TONS,YIELD in TONS / ACRE,PRICE RECEIVED in $ / BU
10,2000,ILLINOIS,17,,,YEAR,CORN,,,,11200000,,,,
11,2000,INDIANA,18,,,YEAR,CORN,,,,5700000,,,,
12,2000,IOWA,19,,,YEAR,CORN,,,,12300000,,,,
13,2000,KANSAS,20,,,YEAR,CORN,,,,3450000,,,,
20,2000,MINNESOTA,27,,,YEAR,CORN,,,,7200000,,,,


In [131]:
# Create a clean copy of the data
cleaned_df = copy.deepcopy(data)

# Filter for corn grain only
cleaned_df = cleaned_df[cleaned_df['COMMODITY'] == "CORN, GRAIN"]

# Create separate dataframes for year and marketing year rows
year_df = cleaned_df[cleaned_df['REFERENCE PERIOD'] == 'YEAR'].copy()
marketing_year_df = cleaned_df[cleaned_df['REFERENCE PERIOD'] == 'MARKETING YEAR'].copy()

# Create a mapping of (location, year) to price
price_mapping = marketing_year_df.set_index(['LOCATION', 'YEAR'])['PRICE RECEIVED in $ / BU']

# Map the prices to the year rows using location and year as keys
year_df['PRICE RECEIVED in $ / BU'] = year_df.apply(
    lambda row: price_mapping.get((row['LOCATION'], row['YEAR']), None), 
    axis=1
)

# This will be our final dataframe
cleaned_df = year_df

# Drop unnecessary columns
cleaned_df = cleaned_df.drop(["PRODUCTION in TONS", "COMMODITY", "ASD CODE", "COUNTY ANSI", "AREA PLANTED in ACRES", "YIELD in TONS / ACRE", "STATE ANSI"], axis=1)

In [132]:
cleaned_df.head()

,YEAR,LOCATION,REFERENCE PERIOD,AREA HARVESTED in ACRES,PRODUCTION in BU,YIELD in BU / ACRE,PRODUCTION in $,PRICE RECEIVED in $ / BU
1210,2000,ALABAMA,YEAR,"165,000","10,725,000",65,"23,166,000",2.16
1212,2000,ARIZONA,YEAR,"33,000","6,468,000",196,"17,981,000",2.78
1214,2000,ARKANSAS,YEAR,"175,000","22,750,000",130,"39,813,000",1.75
1216,2000,CALIFORNIA,YEAR,"205,000","34,850,000",170,"85,034,000",2.44
1218,2000,COLORADO,YEAR,"1,150,000","144,900,000",126,"301,392,000",2.08


In [136]:
# Convert both columns to numeric
cleaned_df['AREA HARVESTED in ACRES'] = pd.to_numeric(
    cleaned_df['AREA HARVESTED in ACRES'].astype(str).str.replace(',', ''),
    errors='coerce'
)

cleaned_df['PRICE RECEIVED in $ / BU'] = pd.to_numeric(
    cleaned_df['PRICE RECEIVED in $ / BU'],
    errors='coerce'
)

# Group by location and calculate mean for numeric columns
location_averages = cleaned_df.groupby('LOCATION').agg({
    'AREA HARVESTED in ACRES': 'mean',
    'PRICE RECEIVED in $ / BU': 'mean'
}).round(2)

# Sort by area harvested to see largest producers first
location_averages = location_averages.sort_values('AREA HARVESTED in ACRES', ascending=False)

# Display the results
print("Number of locations:", len(location_averages))
location_averages

Number of locations: 42


,AREA HARVESTED in ACRES,PRICE RECEIVED in $ / BU
LOCATION,,
IOWA,12758000.0,3.88
ILLINOIS,11400000.0,3.90
NEBRASKA,8879200.0,3.88
MINNESOTA,7389200.0,3.75
INDIANA,5478400.0,4.00
SOUTH DAKOTA,4584800.0,3.68
KANSAS,4158800.0,3.92
OHIO,3240400.0,3.98
MISSOURI,3079200.0,3.96


In [139]:
# Filter for locations with >= 1 million acres harvested
major_producers = location_averages[location_averages['AREA HARVESTED in ACRES'] >= 4_000_000]

# Display the results
print("Number of major producing locations:", len(major_producers))
major_producers

Number of major producing locations: 7


,AREA HARVESTED in ACRES,PRICE RECEIVED in $ / BU
LOCATION,,
IOWA,12758000.0,3.88
ILLINOIS,11400000.0,3.90
NEBRASKA,8879200.0,3.88
MINNESOTA,7389200.0,3.75
INDIANA,5478400.0,4.00
SOUTH DAKOTA,4584800.0,3.68
KANSAS,4158800.0,3.92
